In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

import sys
import tempfile
import itertools

import tensorflow as tf

from categorical_embedding_tf_train import loaddata, streaming_mape, input_fn
from categorical_embedding_tf_model import build_estimator

In [2]:
sales = pd.read_csv('sales.txt',
                    delimiter=';',
                    nrows=None,
                    parse_dates=['retailweek'],
                    date_parser=(lambda dt: pd.to_datetime(dt, format='%Y-%m-%d')))

attributes = pd.read_csv('article_attributes.txt', delimiter=';')


sales['saleslog'] = np.log1p(sales['sales'])
sales['saleslog'] = sales['saleslog'].astype(np.float32)

features_x = ['regular_price', 'current_price', 'ratio', 'promo1', 'promo2']

var_name = 'retailweek'

sales['day'] = pd.Index(sales[var_name]).day
sales['week'] = pd.Index(sales[var_name]).week
sales['month'] = pd.Index(sales[var_name]).month
sales['year'] = pd.Index(sales[var_name]).year
sales['dayofyear'] = pd.Index(sales[var_name]).dayofyear

sales['day'] = sales['day'].fillna(0)
sales['week'] = sales['week'].fillna(0)
sales['month'] = sales['month'].fillna(0)
sales['year'] = sales['year'].fillna(0)
sales['dayofyear'] = sales['dayofyear'].fillna(0)

features_x.append('day')
features_x.append('week')
features_x.append('month')
features_x.append('year')
features_x.append('dayofyear')

# join the two dataframes
data_merged = pd.merge(sales, attributes, on='article')

for col in list(data_merged.select_dtypes(include=['object']).columns):
    #     data_merged[col+'_numeric'] = pd.Categorical(data_merged[col])
    #     data_merged[col+'_numeric'] = data_merged[col+'_numeric'].cat.codes
    #     features_x.append(col+'_numeric')
    features_x.append(col)

for col in ['cost',
            'rgb_r_main_col', 'rgb_g_main_col', 'rgb_b_main_col',
            'rgb_r_sec_col', 'rgb_g_sec_col', 'rgb_b_sec_col']:
    features_x.append(col)

data_merged.promo1 = data_merged.promo1.astype(str)
data_merged.promo2 = data_merged.promo2.astype(str)

df_train = data_merged[data_merged['retailweek'] < '2017-03-31']
df_test = data_merged[data_merged['retailweek'] > '2017-03-31']

In [3]:
import math

for col in data_merged.columns:
    print col+': '+str(len(data_merged[col].unique()))+': '+str(math.ceil(math.log(len(data_merged[col].unique()), 2)))

country: 3: 2.0
article: 476: 9.0
sales: 785: 10.0
regular_price: 123: 7.0
current_price: 174: 8.0
ratio: 5154: 13.0
retailweek: 123: 7.0
promo1: 2: 1.0
promo2: 2: 1.0
saleslog: 785: 10.0
day: 31: 5.0
week: 53: 6.0
month: 12: 4.0
year: 4: 2.0
dayofyear: 123: 7.0
productgroup: 6: 3.0
category: 21: 5.0
cost: 118: 7.0
style: 3: 2.0
sizes: 8: 3.0
gender: 4: 2.0
rgb_r_main_col: 49: 6.0
rgb_g_main_col: 75: 7.0
rgb_b_main_col: 65: 7.0
rgb_r_sec_col: 49: 6.0
rgb_g_sec_col: 75: 7.0
rgb_b_sec_col: 65: 7.0


In [10]:
list_articles = list(data_merged['article'].unique())
list_productgroup = list(data_merged['productgroup'].unique())
list_category = list(data_merged['category'].unique())
list_sizes = list(data_merged['sizes'].unique())

In [11]:
import pickle
with open('list_articles.txt', 'wb') as f:
    pickle.dump(list_articles, f)
    
with open('list_productgroup.txt', 'wb') as f:
    pickle.dump(list_productgroup, f)
    
with open('list_category.txt', 'wb') as f:
    pickle.dump(list_category, f)
    
with open('list_sizes.txt', 'wb') as f:
    pickle.dump(list_sizes, f)

In [13]:
data_merged['category'].unique()

array(['RUNNING', 'TRAINING', 'TENNIS', 'FOOTBALL GENERIC', 'OUTDOOR',
       'INDOOR', 'STELLA', 'FOOTBALL LICENSED', 'SWIM', 'GOLF',
       'ACTION SPORTS', 'RELAX CASUAL', 'CLASSICS', 'DRESS CASUAL',
       'ORIGINALS', 'PORSCHE', 'BASKETBALL GENERIC', 'SLVR',
       'ACTIVE CASUAL', 'OLYMPICS', 'NEO'], dtype=object)